# Notebook to calculate CW-SSIM, PSNR, FID, and other Pytorch metrics:

### Calculate CW-SSIM for GT vs pix2pix, GT vs I2SB:
### from repo: https://github.com/jterrace/pyssim/tree/master

In [3]:
from ssim import SSIM
from PIL import Image, ImageOps
import os
from natsort import natsorted
import numpy as np
from tqdm import tqdm

In [4]:
GT_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE"
p2p_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\pix2pix_sample1_only\val"
i2sb_img_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\unconditional\test-run-4\iter_24388"
GT_img_list = [os.path.join(GT_img_pth,x) for x in os.listdir(GT_img_pth) if x.endswith(".png")]
p2p_img_list = [os.path.join(p2p_img_pth,x) for x in os.listdir(p2p_img_pth) if x.endswith(".png")]
i2sb_img_list = [os.path.join(i2sb_img_pth,x) for x in os.listdir(i2sb_img_pth) if x.endswith(".png")]
GT_img_list = natsorted(GT_img_list)
p2p_img_list = natsorted(p2p_img_list)
i2sb_img_list = natsorted(i2sb_img_list)
GT_img_list = GT_img_list[:5952] #sample1, sample2A
p2p_img_list = p2p_img_list[:5952]

### Pix2Pix CWSSIM score:

In [3]:
pix2pix_cwssim_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(p2p_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    pix2pix_cwssim_total += SSIM(ref_img).cw_ssim_value(test_img)
print("Average CWSSIM score for Pix2Pix is {}".format(pix2pix_cwssim_total // len(GT_img_list)))

Images Processed: 100%|██████████| 5952/5952 [18:33<00:00,  5.34it/s]  

Average CWSSIM score for Pix2Pix is 0.0


In [8]:
pix2pix_cwssim_total/5952

0.5002800103454716

### I2SB CWSSIM score:

In [4]:
i2sb_cwssim_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(i2sb_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    i2sb_cwssim_total += SSIM(ref_img).cw_ssim_value(test_img)
print("Average CWSSIM score for I2SB is {}".format(i2sb_cwssim_total // len(GT_img_list)))

Images Processed: 100%|██████████| 5952/5952 [15:57<00:00,  6.21it/s]

Average CWSSIM score for I2SB is 0.0


In [7]:
i2sb_cwssim_total/5952

0.5174699244136098

# Calculate PSNR for GT vs pix2pix, GT vs I2SB:

In [11]:
def calculate_psnr(img1, img2, max_value=255):
    mse = np.mean((np.array(img1, dtype=np.float32) - np.array(img2, dtype=np.float32)) ** 2)
    if mse == 0:
        return 100
    return 20 * np.log10(max_value / (np.sqrt(mse)))

### Pix2Pix PSNR Score:

In [14]:
pix2pix_psnr_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(p2p_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    pix2pix_psnr_total += calculate_psnr(ref_img, test_img)
print("Average PSNR score for Pix2Pix is {}".format(pix2pix_psnr_total / len(GT_img_list)))

Images Processed: 100%|██████████| 5952/5952 [02:36<00:00, 37.92it/s]

Average PSNR score for Pix2Pix is 14.566912677808372


### I2SB PSNR Score:

In [17]:
i2sb_psnr_total = 0
for idx, images in tqdm(enumerate(GT_img_list),colour='red',desc='Images Processed', total = len(GT_img_list)):
    ref_img = Image.open(GT_img_list[idx])
    test_img = Image.open(i2sb_img_list[idx])
    ref_img = ImageOps.grayscale(ref_img)
    test_img = ImageOps.grayscale(test_img)
    i2sb_psnr_total += calculate_psnr(ref_img, test_img)
print("Average PSNR score for I2SB is {}".format(i2sb_psnr_total / len(GT_img_list)))

Images Processed: 100%|██████████| 5952/5952 [03:26<00:00, 28.85it/s]

Average PSNR score for I2SB is 14.40829158977186


# Calculate FID for GT vs pix2pix, GT vs I2SB:
### from repo: https://github.com/mseitzer/pytorch-fid

### Pix2Pix FID Score:

In [18]:
!python -m pytorch_fid \\shelter\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\tmp\GT \\shelter\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\tmp\pix2pix_sample1_only --device cuda:0

FID:  25.728143461570426



100%|##########| 120/120 [00:43<00:00,  2.78it/s]

100%|##########| 120/120 [00:26<00:00,  4.49it/s]


### I2SB FID Score

In [19]:
!python -m pytorch_fid \\shelter\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\tmp\GT \\shelter\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\tmp\iter_24388 --device cuda:0

FID:  6.961478786222131



100%|##########| 120/120 [00:37<00:00,  3.24it/s]

100%|##########| 120/120 [00:28<00:00,  4.17it/s]


### Trying DISTS (lower the better), LPIPS (lower the better), style loss (lower the better), VSI (0-1, higher the better), and haarpsi (0-1, higher the better) as a metric:

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from piq import DISTS, LPIPS, vsi, haarpsi, StyleLoss
from torchvision.transforms import ToTensor
import torchvision.transforms as T

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_transform = T.Compose([ToTensor()]) # [0,255] -> [0,1], HWC -> CHW

### For Pix2Pix (Batch Size doesn't change the results, higher batch size for faster computation):

In [9]:
class TestDataset(Dataset):
    def __init__(self, GT_img_list, test_img_list, transforms=None):
        self.GT_img_list = GT_img_list
        self.test_img_list = test_img_list
        self.transforms = transforms

    def __len__(self):
        if len(self.GT_img_list) != len(self.test_img_list):
            assert("Length of Test and GT Image List Unequal")
        return len(GT_img_list)

    def __getitem__(self,idx):
        ref_img = np.array(Image.open(self.GT_img_list[idx]))
        test_img = np.array(Image.open(self.test_img_list[idx]))
        if self.transforms is not None:
            ref_img = self.transforms(ref_img)
            test_img = self.transforms(test_img)
        return ref_img, test_img

In [12]:
p2p_img_dataset = TestDataset(GT_img_list= GT_img_list, test_img_list= p2p_img_list, transforms = tensor_transform)
p2p_img_dataloader = DataLoader(dataset=p2p_img_dataset,
                                  batch_size=32,
                                  #pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
#check
ref_img_batch, p2p_img_batch = next(iter(p2p_img_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(ref_img_batch.size(), p2p_img_batch.size()))

Images have a tensor size of torch.Size([32, 3, 256, 256]), and Labels have a tensor size of torch.Size([32, 3, 256, 256])


In [13]:
pix2pix_dists_total = 0
pix2pix_lpips_total = 0
pix2pix_vsi_total = 0
pix2pix_haarpsi_total = 0
pix2pix_style_loss_total = 0
pbar = tqdm(enumerate(p2p_img_dataloader), total=len(p2p_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, p2p_img) in pbar:
    ref_img = ref_img.to(device)
    p2p_img = p2p_img.to(device)
    pix2pix_vsi_total += vsi(p2p_img,ref_img)
    pix2pix_dists_total += DISTS()(p2p_img,ref_img)
    pix2pix_lpips_total += LPIPS()(p2p_img,ref_img)
    pix2pix_haarpsi_total += haarpsi(p2p_img,ref_img)
    pix2pix_style_loss_total += StyleLoss()(p2p_img,ref_img)
print("Average VSI score for Pix2Pix is {}".format(pix2pix_vsi_total / len(p2p_img_dataloader)))
print("Average DISTS score for Pix2Pix is {}".format(pix2pix_dists_total / len(p2p_img_dataloader)))
print("Average LPIPS score for Pix2Pix is {}".format(pix2pix_lpips_total / len(p2p_img_dataloader)))
print("Average HAARPSI score for Pix2Pix is {}".format(pix2pix_haarpsi_total / len(p2p_img_dataloader)))
print("Average Style score for Pix2Pix is {}".format(pix2pix_style_loss_total / len(p2p_img_dataloader)))

Images Processed: 100%|██████████| 186/186 [11:24<00:00,  3.68s/it]

Average VSI score for Pix2Pix is 0.834528923034668
Average DISTS score for Pix2Pix is 0.2199782282114029
Average LPIPS score for Pix2Pix is 0.4710122048854828
Average HAARPSI score for Pix2Pix is 0.35326284170150757
Average Style score for Pix2Pix is 20185016.0


### For I2SB:

In [15]:
i2sb_img_dataset = TestDataset(GT_img_list= GT_img_list, test_img_list= i2sb_img_list, transforms = tensor_transform)
i2sb_img_dataloader = DataLoader(dataset=i2sb_img_dataset,
                                  batch_size=32,
                                  #pin_memory= true allows faster data transport from cpu to gpu
                                  num_workers=0, pin_memory=True, shuffle=False)
#check:
ref_img_batch, i2sb_img_batch = next(iter(i2sb_img_dataloader))
print("Images have a tensor size of {}, and Labels have a tensor size of {}".
      format(ref_img_batch.size(), i2sb_img_batch.size()))

Images have a tensor size of torch.Size([32, 3, 256, 256]), and Labels have a tensor size of torch.Size([32, 3, 256, 256])


In [16]:
i2sb_dists_total = 0
i2sb_lpips_total = 0
i2sb_vsi_total = 0
i2sb_haarpsi_total = 0
i2sb_style_loss_total = 0
pbar = tqdm(enumerate(i2sb_img_dataloader), total=len(i2sb_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_img = i2sb_img.to(device)
    i2sb_vsi_total += vsi(i2sb_img,ref_img)
    i2sb_dists_total += DISTS()(i2sb_img,ref_img)
    i2sb_lpips_total += LPIPS()(i2sb_img,ref_img)
    i2sb_haarpsi_total += haarpsi(i2sb_img,ref_img)
    i2sb_style_loss_total += StyleLoss()(i2sb_img,ref_img)
print("Average VSI score for I2SB is {}".format(i2sb_vsi_total / len(i2sb_img_dataloader)))
print("Average DISTS score for I2SB is {}".format(i2sb_dists_total / len(i2sb_img_dataloader)))
print("Average LPIPS score for I2SB is {}".format(i2sb_lpips_total / len(i2sb_img_dataloader)))
print("Average HAARPSI score for I2SB is {}".format(i2sb_haarpsi_total / len(i2sb_img_dataloader)))
print("Average Style score for I2SB is {}".format(i2sb_style_loss_total / len(i2sb_img_dataloader)))

Images Processed:   0%|          | 0/186 [00:00<?, ?it/s]C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kevin\.conda\envs\wsi_analysis1\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Images Processed: 100%|██████████| 186/186 [11:32<00:00,  3.72s/it]

Average VSI score for I2SB is 0.8354562520980835
Average DISTS score for I2SB is 0.18133263289928436
Average LPIPS score for I2SB is 0.4468766748905182
Average HAARPSI score for I2SB is 0.3566141426563263
Average Style score for I2SB is 7872890.5


In [18]:
### VSI and HAARPSI, like PSNR and SSIM don't seem to be able to differentiate the two! While DISTS LPIPS and Style, which are perceptual based using VGG16, kind of similar to FID, are able to differentiate the two!

In [28]:
from piq import PieAPP, ssim

In [30]:
pix2pix_pieapp_total = 0
pix2pix_ssim_total = 0
pbar = tqdm(enumerate(p2p_img_dataloader), total=len(p2p_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, p2p_img) in pbar:
    ref_img = ref_img.to(device)
    p2p_img = p2p_img.to(device)
    pix2pix_pieapp_total += PieAPP()(p2p_img,ref_img)
    pix2pix_ssim_total += ssim(p2p_img,ref_img)
print("Average PieAPP score for Pix2Pix is {}".format(pix2pix_pieapp_total / len(p2p_img_dataloader)))
print("Average SSIM score for Pix2Pix is {}".format(pix2pix_ssim_total / len(p2p_img_dataloader)))


Images Processed: 100%|██████████| 186/186 [22:16<00:00,  7.19s/it]

Average PieAPP score for Pix2Pix is 2.0399372577667236
Average SSIM score for Pix2Pix is 0.18936103582382202


In [31]:
i2sb_pieapp_total = 0
i2sb_ssim_total = 0
pbar = tqdm(enumerate(i2sb_img_dataloader), total=len(i2sb_img_dataloader), desc='Images Processed', colour='red')
for idx, (ref_img, i2sb_img) in pbar:
    ref_img = ref_img.to(device)
    i2sb_img = i2sb_img.to(device)
    i2sb_pieapp_total += PieAPP()(i2sb_img,ref_img)
    i2sb_ssim_total += ssim(i2sb_img,ref_img)
print("Average PieAPP score for I2SB is {}".format(i2sb_pieapp_total / len(i2sb_img_dataloader)))
print("Average SSIM score for I2SB is {}".format(i2sb_ssim_total / len(i2sb_img_dataloader)))


Images Processed: 100%|██████████| 186/186 [20:08<00:00,  6.50s/it]

Average PieAPP score for I2SB is 1.549119234085083
Average SSIM score for I2SB is 0.18343544006347656
